In [1]:
import requests
import datetime as dt
import time

In [4]:
date = dt.date(2019, 1, 9)
season_type='Regular Season'
season = '2018-19'

In [9]:
USER_AGENT=('Mozilla/5.0 (Windows NT 10.0; Win64; x64) ' +
    'AppleWebKit/537.36 (KHTML, like Gecko) ' +
    'Chrome/70.0.3538.77 Safari/537.36')

REQUEST_HEADERS = {'user-agent': USER_AGENT}

NBA_ID = '00'
NBA_URL = 'https://stats.nba.com/stats/teamgamelogs'
date_from = date
date_to = date_from
date_from_string = date_from.strftime('%m/%d/%Y')
date_to_string = date_from_string

nba_params = {
    'LeagueID': NBA_ID,
    'Season': season,
    'SeasonType': season_type,
    'DateFrom': date_from_string, 
    'DateTo': date_to_string,
    'Outcome': '' # Can set to 'W' or 'L' to just get winners or losers
}

print('Accessing stats.nba.com for games on {}'.format(date), end='.....')

time.sleep(5)
r = requests.get(NBA_URL, params=nba_params, headers=REQUEST_HEADERS, 
    allow_redirects=False, timeout=15)
assert r.status_code == 200
print('Successful')

Accessing stats.nba.com for games on 2019-01-09.....Successful


In [15]:
nba_params

{'DateFrom': '01/09/2019',
 'DateTo': '01/09/2019',
 'LeagueID': '00',
 'Outcome': '',
 'Season': '2018-19',
 'SeasonType': 'Regular Season'}

In [10]:
json_dict = r.json() # Turns the json text into a python dict
games_dict = json_dict['resultSets'][0] # Only has one element
headers = games_dict['headers']

In [ ]:
GAME_ID_COL = headers.index('GAME_ID')
DATE_COL = headers.index('GAME_DATE')
MATCHUP_COL = headers.index('MATCHUP')
games_list = games_dict['rowSet'] # List of list

df_games = pd.DataFrame(columns=headers, data=games_list)
df_games = df_games[['SEASON_YEAR', 'TEAM_ID', 'TEAM_ABBREVIATION',
                     'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL',
                     'PTS']]
df_games['HOME_AWAY'] = df_games['MATCHUP'].apply(lambda x: 'HOME' if  'vs.' in x else 'AWAY')

# Get the opponent's info
df_games['TEAM_ABBREVIATION_OPP'] = df_games['MATCHUP'].apply(lambda x: x[-3:])
df_games = pd.merge(df_games, df_games[['TEAM_ID', 'TEAM_ABBREVIATION', 'PTS']], 
                    left_on='TEAM_ABBREVIATION_OPP', right_on = 'TEAM_ABBREVIATION',
                    suffixes=('', '_OPP'))
df_games = df_games.rename(columns={'TEAM_ABBREVIATION_OPP': 'OPP_TEAM_ABBREVIATION',
                         'PTS_OPP': 'OPP_PTS',
                         'TEAM_ID_OPP': 'OPP_TEAM_ID'
                        })

# Remove duplicate column of 'OPP_TEAM_ABBREVIATION'
df_games = df_games.loc[:,~df_games.columns.duplicated()]

if len(df_games) == 0:
    print('No games on {}'.format(date))